In [ ]:
"""
Gan ・・・・・ use Close price
"""

In [3]:
import schedule
import time
# from job import job
from job3 import job
import pandas as pd
import datetime
import numpy as np
import rpy2
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from DWX_ZeroMQ_Connector_v2_0_1_RC8 import DWX_ZeroMQ_Connector

%load_ext rpy2.ipython

In [ ]:
%%R
library(tidyverse)
library(quantmod)
# file_path <- "C:\\Users\\komoo\\AppData\\Roaming\\MetaQuotes\\Terminal\\A0BA7E3FF12928487EFBA3BA88FE6F94\\MQL4\\Files\\Export_History\\AUDUSD\\AUDUSD_M15.csv"
# file_path <- "C:\\Users\\komoo\\AppData\\Roaming\\MetaQuotes\\Terminal\\A0BA7E3FF12928487EFBA3BA88FE6F94\\MQL4\\Files\\Export_History\\EURUSD\\EURUSD_M1.csv"
# file_path <- "C:\\Users\\komoo\\AppData\\Roaming\\MetaQuotes\\Terminal\\A0BA7E3FF12928487EFBA3BA88FE6F94\\MQL4\\Files\\Export_History\\EURJPY\\EURJPY_M1.csv"
# file_path <- "C:\\Users\\komoo\\AppData\\Roaming\\MetaQuotes\\Terminal\\A0BA7E3FF12928487EFBA3BA88FE6F94\\MQL4\\Files\\Export_History\\USDJPY\\USDJPY_M15.csv"
file_path <- "C:\\Users\\komoo\\AppData\\Roaming\\MetaQuotes\\Terminal\\A0BA7E3FF12928487EFBA3BA88FE6F94\\MQL4\\Files\\Export_History\\GBPJPY\\GBPJPY_M5.csv"

load_csv <- function(){
  dat <- read_csv(file_path) %>% mutate(close=(High+Low+Close+Close)/4,wma=EMA(close,4),wma2=EMA(close,20), wma3=EMA(close,10),rsi=RSI(wma2,2)) %>% filter(!is.na(wma2)) %>% write_csv("audpred.csv")
}

In [ ]:
model1 = tf.keras.models.load_model("Close.hdf5")

In [ ]:
def load_csv():
    %R dat <- read_csv(file_path) %>% mutate(wma = WMA(Close,2)) %>% filter(!is.na(wma)) #%>% mutate(wma = SMA(wma,2)) %>% filter(!is.na(wma))
#     %R dat <- read_csv(file_path) %>% mutate(rsi = RSI(Close,70),rsi=EMA(rsi,4)) %>% filter(!is.na(rsi))
#     %R st <- TTR::stoch(dat$Close,60,2,2,maType="WMA") %>% as_tibble() %>% mutate(Open=dat$Open,rsi=dat$rsi) %>% filter(!is.na(slowD))
    %R write_csv(dat,"audpred.csv")

In [ ]:
file_path = "C:\\Users\\komoo\\AppData\\Roaming\\MetaQuotes\\Terminal\\A0BA7E3FF12928487EFBA3BA88FE6F94\\MQL4\\Files\\Export_History\\GBPJPY\\USDJPY_M5.csv"
def trade_run(lot_size=0.01):
    date2 = datetime.datetime.now()
    print(date2)
    a = True
    while a == True:
        try:
            time.sleep(1.01)
            load_csv()
            df = pd.read_csv("audpred.csv")
            x = df[["Close"]]
            a = False
        except:
            pass

    fit = MinMaxScaler().fit(x)
    x = MinMaxScaler().fit_transform(x)
    
    x = x[-200::]
#     predict_generator
    gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x,x,16,stride=1, batch_size=6000)
    pred1 = model1.predict_generator(gen)
    
    op = np.asanyarray(df[["Open"]]).reshape((-1,1))
    job(x,pred1,op, symbol="GBPJPY",loscut=20,sl=100,tp=2000, lot_size=lot_size,use_pred=True)
    date2 = datetime.datetime.now()
    print(date2)

In [ ]:
from IPython.display import clear_output

def clear():
    time.sleep(2.5)
    clear_output()
    
def pip_clear():
    _zmq = DWX_ZeroMQ_Connector()
    _zmq._DWX_MTX_CLOSE_ALL_TRADES_()
    raise TypeError("時間です")

In [ ]:
trade_run()

In [ ]:
# schedule.clear()
# schedule.every().hour.at("00:00").do(trade_run)
# schedule.every().hour.at("00:05").do(trade_run)
# schedule.every().hour.at("00:10").do(trade_run)
# schedule.every().hour.at("00:15").do(trade_run)
# schedule.every().hour.at("00:20").do(trade_run)
# schedule.every().hour.at("00:25").do(trade_run)
# schedule.every().hour.at("00:30").do(trade_run)
# schedule.every().hour.at("00:35").do(trade_run)
# schedule.every().hour.at("00:40").do(trade_run)
# schedule.every().hour.at("00:45").do(trade_run)
# schedule.every().hour.at("00:50").do(trade_run)
# schedule.every().hour.at("00:55").do(trade_run)
schedule.every().minute.at(":00").do(trade_run)
schedule.every().hour.at("00:01").do(clear)
# schedule.every().day.at("05:53").do(pip_clear)
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
def run():
    schedule.clear()
    schedule.every().minute.at(":00").do(trade_run)
    while True:
      schedule.run_pending()
      time.sleep(1)

In [ ]:
schedule.every().monday.at("06:00").do(run)
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
import calendar
weekday = datetime.date.today().weekday()
weekday_name = calendar.day_name[weekday]

date2 = datetime.datetime.now()


In [ ]:
load_csv()

In [ ]:
x = df[["wma"]]
x = np.asanyarray(x)

In [ ]:
s = np.append(x,x[-1])